<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Prophet_automatic_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import itertools
import numpy as np
import pandas as pd
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics

In [15]:
df = pd.read_csv('drive/My Drive/Churn/same_day_vol_port_basic_cdma_aug.csv')

In [16]:
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

In [17]:
# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = [] # Store the RMSEs for each params here

In [18]:
cutoffs = pd.to_datetime(['2019-01-01', '2019-06-01', '2020-01-01'])

In [20]:
# Use cross validation to evaluate all parameters
for params in all_params:
  m = Prophet(**params).fit(df) # Fit the model with given params
  df_cv = cross_validation(m, initial = '761 days', period = '180 days', horizon = '30 days')
  df_p = performance_metrics(df_cv, rolling_window = 1)
  rmses.append(df_p['rmse'].values[0])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 4 forecasts with cutoffs between 2019-02-01 00:00:00 and 2020-07-25 00:00:00
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 4 forecasts with cutoffs between 2019-02-01 00:00:00 and 2020-07-25 00:00:00
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 4 forecasts with cutoffs between 2019-02-01 00:00:00 and 2020-07-25 00:00:00
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 4 forecasts with cutoffs between 2019-02-01 00:00:00 and 2020-07-25 00:00:00
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 4 forecasts with cutoffs between 2019-02-01 00:00:00 and 2020-07-25 00:0

In [21]:
# Find the best results
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

    changepoint_prior_scale  seasonality_prior_scale       rmse
0                     0.001                     0.01  12.222572
1                     0.001                     0.10  12.286415
2                     0.001                     1.00  12.022563
3                     0.001                    10.00  12.165894
4                     0.010                     0.01  11.308660
5                     0.010                     0.10  11.753322
6                     0.010                     1.00  11.799704
7                     0.010                    10.00  11.740697
8                     0.100                     0.01  11.397293
9                     0.100                     0.10  11.734062
10                    0.100                     1.00  11.729134
11                    0.100                    10.00  11.735391
12                    0.500                     0.01  11.884006
13                    0.500                     0.10  11.886305
14                    0.500             

In [22]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
